In [1]:
import deepdoctection as dd
from IPython.core.display import HTML
from matplotlib import pyplot as plt
import os

analyzer = dd.get_dd_analyzer()  # instantiate the built-in analyzer similar to the Hugging Face space demo

comp=analyzer.pipe_component_list
print(comp)
comp[7]=(dd.TextOrderService(text_container="word",
                                 text_block_categories= [dd.LayoutType.text,
                                                         dd.LayoutType.title,
                                                         dd.LayoutType.list,
                                                         dd.LayoutType.table],
                                 floating_text_block_categories=[dd.LayoutType.text,
                                                         dd.LayoutType.title,
                                                         dd.LayoutType.list,
                                                         dd.LayoutType.table],
                                 include_residual_text_container=True))

page_parsing = dd.PageParsingService(text_container="word",
                                     floating_text_block_categories=[dd.LayoutType.text,
                                                                     dd.LayoutType.title,
                                                                     dd.LayoutType.list,
                                                                     dd.LayoutType.table],
                                     include_residual_text_container=True)

pipe = dd.DoctectionPipe(pipeline_component_list=comp,
                         page_parsing_service=page_parsing)



[0109 21:16.05 @file_utils.py:36]  INF  PyTorch version 1.12.0 available.
[0109 21:16.10 @dd.py:422]  INF  Config: 
 {'DEVICE': 'cpu',
 'LANGUAGE': None,
 'LAYOUT_NMS_PAIRS': {'COMBINATIONS': None, 'PRIORITY': None, 'THRESHOLDS': None},
 'LIB': 'PT',
 'OCR': {'CONFIG': {'TESSERACT': 'dd/conf_tesseract.yaml'},
         'USE_DOCTR': False,
         'USE_TESSERACT': True,
         'USE_TEXTRACT': False,
         'WEIGHTS': {'DOCTR_RECOGNITION': {'PT': 'doctr/crnn_vgg16_bn/pt/crnn_vgg16_bn-9762b0b0.pt',
                                           'TF': 'doctr/crnn_vgg16_bn/tf/crnn_vgg16_bn-76b7f2c6.zip'},
                     'DOCTR_WORD': {'PT': 'doctr/db_resnet50/pt/db_resnet50-ac60cadc.pt',
                                    'TF': 'doctr/db_resnet50/tf/db_resnet50-adcafc63.zip'}}},
 'PT': {'CELL': {'FILTER': None,
                 'WEIGHTS': 'cell/d2_model_1849999_cell_inf_only.pt',
                 'WEIGHTS_TS': 'cell/d2_model_1849999_cell_inf_only.ts'},
        'ITEM': {'FILTER': None

[<deepdoctection.pipe.layout.ImageLayoutService object at 0x7f057b004b80>, <deepdoctection.pipe.cell.SubImageLayoutService object at 0x7f057b01a440>, <deepdoctection.pipe.cell.SubImageLayoutService object at 0x7f057af96920>, <deepdoctection.pipe.segment.TableSegmentationService object at 0x7f057af94f10>, <deepdoctection.pipe.refine.TableSegmentationRefinementService object at 0x7f057af962c0>, <deepdoctection.pipe.text.TextExtractionService object at 0x7f057af97fd0>, <deepdoctection.pipe.common.MatchingService object at 0x7f057af97d30>, <deepdoctection.pipe.order.TextOrderService object at 0x7f057af979d0>]


In [2]:
path = "./sample/image.pdf"
df = pipe.analyze(path=path)
df.reset_state() 

# Create a directory to save the pages if it doesn't exist
output_dir = "./sample/pages"
os.makedirs(output_dir, exist_ok=True)
dpts = []
layout_items_str = ""

# Iterate through all the pages
for dp in df:
    dpts.append(dp)
    out = dp.as_dict()
    layout_items = [layout for layout in dp.layouts if layout.reading_order is not None]
    layout_items.sort(key=lambda x: x.reading_order)
    layout_items_str += f"\n\n -------- PAGE NUMBER: {dp.page_number+1} ------------- \n"
    for item in layout_items:
        layout_items_str += f"\n {item.category_name}: {item.text}"  


print(layout_items_str)

[0109 21:16.15 @doctectionpipe.py:84]  INF  Processing image_0.pdf
/home/lenovo/.local/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
[0109 21:16.21 @context.py:126]  INF  ImageLayoutService total: 2.8138 sec.
[0109 21:16.21 @context.py:126]  INF  SubImageLayoutService total: 0.0 sec.
[0109 21:16.21 @context.py:126]  INF  SubImageLayoutService total: 0.0 sec.
[0109 21:16.21 @context.py:126]  INF  TableSegmentationService total: 0.0 sec.
[0109 21:16.21 @context.py:126]  INF  TableSegmentationRefinementService total: 0.0 sec.
[0109 21:16.22 @context.py:126]  INF  TextExtractionService total: 1.2088 sec.
[0109 21:16.22 @context.py:126]  INF  MatchingService total: 0.0041 sec.
[0109 21:16.22 @context.py:126]  INF  TextOrderService total: 0



 -------- PAGE NUMBER: 1 ------------- 

 title: Title: The Power of Education: Empowering Minds, Transforming Lives
 title: Introduction:
 text: Education plays a vital role in shaping individuals and societies, serving as a catalyst for personal and societal growth. It equips individuals with knowledge, skills, and values necessary to navigate the complexities of the world. This article explores the transformative power of education, its impact on individuals and communities, and the evolving landscape of education in the 21st century.
 line: oe dt ee a
 line: =
 line: * i
 line: ~~ ae
 line: Frys sv
 line: 2
 line: a
 line: ae)
 line: 1) is oe
 line: a,
 title: Education and Personal Development:
 text: Education is a lifelong journey that empowers individuals to discover their passions, hone their talents, and develop critical thinking skills. It cultivates intellectual curiosity, promotes creativity, and fosters personal growth. Through education, individuals gain the tools to p

In [3]:
import re
import json

data = {}
current_index = 0
mainTitleFlag = False
prev = None

for dp in df:
    layout_items = [layout for layout in dp.layouts if layout.reading_order is not None]
    layout_items.sort(key=lambda x: x.reading_order)
    
    for item in layout_items:
        if item.category_name == "title":
            if mainTitleFlag:
                data[current_index-1] = {"mainTitle": prev.text}
            data[current_index] = {"title": item.text}
            current_index += 1
            mainTitleFlag = True
            prev = item

        elif item.category_name in ["text", "list"]:
            if mainTitleFlag:
                mainTitleFlag = False
            data[current_index] = {"text": item.text}
            current_index += 1 

with open("output.json", 'w') as json_file:
    json.dump(data, json_file, indent=4)

[0109 21:16.55 @doctectionpipe.py:84]  INF  Processing image_0.pdf
[0109 21:17.01 @context.py:126]  INF  ImageLayoutService total: 3.3692 sec.
[0109 21:17.01 @context.py:126]  INF  SubImageLayoutService total: 0.0 sec.
[0109 21:17.01 @context.py:126]  INF  SubImageLayoutService total: 0.0 sec.
[0109 21:17.01 @context.py:126]  INF  TableSegmentationService total: 0.0 sec.
[0109 21:17.01 @context.py:126]  INF  TableSegmentationRefinementService total: 0.0 sec.
[0109 21:17.02 @context.py:126]  INF  TextExtractionService total: 1.2211 sec.
[0109 21:17.02 @context.py:126]  INF  MatchingService total: 0.0009 sec.
[0109 21:17.02 @context.py:126]  INF  TextOrderService total: 0.0098 sec.
[0109 21:17.02 @doctectionpipe.py:84]  INF  Processing image_1.pdf
[0109 21:17.06 @context.py:126]  INF  ImageLayoutService total: 2.8839 sec.
[0109 21:17.06 @context.py:126]  INF  SubImageLayoutService total: 0.0 sec.
[0109 21:17.06 @context.py:126]  INF  SubImageLayoutService total: 0.0 sec.
[0109 21:17.06 @

In [4]:
print("done")

done
